In [18]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.events import AbstractEventListener,EventFiringWebDriver

In [22]:
#run webdriver with chrome
driver = webdriver.Chrome('chromedriver')
url = 'https://www.linkedin.com/'
driver.get(url)
#maximize window
driver.maximize_window()
#sign in
user = driver.find_element(By.ID,'session_key')
user.click()
user.send_keys('daodq2011@gmail.com')
time.sleep(1)
password = driver.find_element(By.ID,'session_password')
password.click()
password.send_keys('thuythuy')
password.send_keys(Keys.ENTER)
time.sleep(3)
#search
button_search = driver.find_element(By.CLASS_NAME,'search-global-typeahead__input.always-show-placeholder')
button_search.clear()
button_search.click()
button_search.send_keys('data')
button_search.send_keys(Keys.ENTER)
time.sleep(3)

#show all
driver.find_element(By.LINK_TEXT,'See all job results in Vietnam').click()
time.sleep(3)

#scroll inside the web-element
def scroll_element():
    list_scroll_element = []
    while True:
        list_find = driver.find_elements(By.CLASS_NAME,'disabled.ember-view.job-card-container__link.job-card-list__title')
        driver.execute_script("arguments[0].scrollIntoView();", list_find[-1])
        time.sleep(1)
        if list_find[-1] in list_scroll_element:
            break
        else:
            list_scroll_element.append(list_find[-1])
            list_find.clear() 
            
def cleaning_string(string):    
    list_temp = string.replace('\n\n','').replace(' \n','').strip().split('\n')
    for i in range(len(list_temp)):
        list_temp[i] = list_temp[i].strip()
        list_temp[i] = ' '.join(list_temp[i].split())
    return list_temp

def extract_data():
    file_html = BeautifulSoup(driver.page_source, 'html.parser')
    filter_div = file_html.findAll('div', class_ = 'flex-grow-1 artdeco-entity-lockup__content ember-view')
    list_result = []
    for row in filter_div:
        list_info = cleaning_string(row.text)
        filter_class = row.find('a', class_ = 'disabled ember-view job-card-container__link job-card-list__title')
        list_info.append('https://www.linkedin.com' + filter_class.get('href'))
        list_result.append(list_info)
    return list_result

scroll_element()
data_first_page = extract_data()
print(f'xong page: 1, số cty tuyển dụng = {len(data_first_page)}')
url_part_one = 'https://www.linkedin.com/jobs/search/?currentJobId=3193049010&keywords=data&start='
count = 0
while True:
    count += 1
    url_part_two = str(25*count)
    url_full = url_part_one + url_part_two
    driver.get(url_full)
    time.sleep(3)
    list_find = driver.find_elements(By.CLASS_NAME,'disabled.ember-view.job-card-container__link.job-card-list__title')
    time.sleep(3)
    if not list_find:
        print('hết data thoát vòng lặp')
        break
    else:
        scroll_element()
        time.sleep(1)
        data_first_page.extend(extract_data())
        print(f"số lượng : {len(data_first_page)}",end  = ' ')

xong page: 1, số cty tuyển dụng = 25
số lượng : 50 số lượng : 75 số lượng : 100 số lượng : 125 số lượng : 150 số lượng : 175 số lượng : 200 số lượng : 225 số lượng : 250 số lượng : 275 số lượng : 300 số lượng : 325 số lượng : 350 số lượng : 375 số lượng : 400 số lượng : 425 số lượng : 450 số lượng : 475 số lượng : 500 số lượng : 525 số lượng : 550 số lượng : 575 số lượng : 600 số lượng : 625 số lượng : 650 số lượng : 675 số lượng : 700 số lượng : 725 số lượng : 750 số lượng : 775 số lượng : 800 số lượng : 825 số lượng : 839 số lượng : 864 số lượng : 886 số lượng : 895 hết data thoát vòng lặp


In [29]:
df =  pd.DataFrame(data_first_page, columns = ['Jobs','Company_name','Local','Url_page'])
df.dropna(inplace = True)
df

,Jobs,Company_name,Local,Url_page
0,Marketing Intern,TRG International,"Ho Chi Minh City, Vietnam Hybrid",https://www.linkedin.com/jobs/view/3202466764/...
1,Senior Digital Marketing Specialist,Trusting Social,"Ho Chi Minh City, Ho Chi Minh City, Vietnam On...",https://www.linkedin.com/jobs/view/3203872388/...
2,Financial Research Analyst,Boston Consulting Group (BCG),"Ho Chi Minh City, Vietnam Hybrid",https://www.linkedin.com/jobs/view/3208645096/...
4,Data Collection Project for English Speaker in...,Appen,"Hố Nai, Dong Nai, Vietnam Remote",https://www.linkedin.com/jobs/view/3199944324/...
5,Data Collection Project for English Speaker in...,Appen,"Thanh Binh, Dong Nai, Vietnam Remote",https://www.linkedin.com/jobs/view/3199949028/...
...,...,...,...,...
890,Fraud Project Management,Shopee,"Ho Chi Minh City, Ho Chi Minh City, Vietnam On...",https://www.linkedin.com/jobs/view/3188380927/...
891,Salesforce Analyst,Air Liquide USA LLC,"Can Tho, Can Tho, Vietnam",https://www.linkedin.com/jobs/view/3207952288/...
892,Partnership Quality Assurance Specialist,TikTok Vietnam,"Ho Chi Minh City, Ho Chi Minh City, Vietnam On...",https://www.linkedin.com/jobs/view/3205627302/...
893,Projects & Strategic Initiatives (Open Level),One Mount,"Hanoi, Hanoi, Vietnam On-site",https://www.linkedin.com/jobs/view/3167461700/...


In [30]:
#cuối cùng convert sang file csv để lưu trữ
df.to_csv(r'C:\Users\daodq\Desktop\dataset\linkedin.csv', index =False)
print('complete')

complete
